In [32]:
#import libraries
import bs4 as bs
import urllib.request
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import re
import pandas as pd
from urllib.request import urlopen


#Given a URL, find the homepage of the website (sometimes you may be given a URL of an inner page)
def findhomepage(website):
    count=0
    homepage=""
    for i in website:
        if i=="/":
            count+=1
        if count!=3:    #the homepage will be present in the url before the 3rd '/' . eg: https:/(1/2www.exeterpremedia.com/3
            homepage+=i
        elif count==3:
            break
    return(homepage)


#Find the links in the home page (and remove duplicates)  &&  Store the links
def getandstorelinks(website):
    homepage=findhomepage(website)
    source = urllib.request.urlopen(website).read()
    soup = bs.BeautifulSoup(source,'lxml')
    links=[]
    for url in soup.find_all('a'):              #finding all anchor tags
        links.append(url.get('href'))        # getting all tags that have href
    d={}                                                #dictionary is created to remove duplicate links
    unique_links=[]
    for i in links:
        if i not in d:
            d[i]=1
            if i and i[0]=="h":                       #to remove unnecessary anchor tags and get only appropriate links i.e links starting with "https"
                unique_links.append(i)
    return(unique_links)


#For each link, get the web page, extract text and store it
def getwebpageandstoretext(website):
    homepage=findhomepage(website)
    unique_links=getandstorelinks(homepage)
    textineachlink=[]
    for eachlink in unique_links:
        if homepage in eachlink:
            source = urllib.request.urlopen(eachlink).read()
            soup = bs.BeautifulSoup(source,'lxml')
            textineachlink.append(soup.text)                                 #using soup.text to extract text from the webpage
    return textineachlink


#Parse the text (remove stop words, punctuation) and generate a list of uni-grams, and bi-grams from the text. We will refer to these as key terms.
def removepunctuationsandstopwords(website):  
    source = urllib.request.urlopen(website).read()
    soup = bs.BeautifulSoup(source,'lxml')
    sentence=soup.text
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(sentence) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)                                #remove stop words
    string=""
    for i in filtered_sentence:
        string=string+" "+i
    string = re.sub(r'[^\w\s]', '', string)                             #remove punctuations
    return string



#Function to generate unigrams
def uni_grams(sentence):
    dic={}
    unigram=[]
    sentence=sentence.split()
    return sentence

#Function to generate bigrams
def bi_grams(sentence):
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    l = re.split(r' [\.\?!][\'"\)\]] *', sentence)
  #print(l)
  #return l
    if len(l[len(l)-1])==0:
        l.pop()
  #return l 
    ans=[]
    for i in l:
        kish=list(i.split())
        for j in range(1,len(kish)):
            str1=kish[j-1]
            str2=kish[j]
            tmp = ""
            for k in str1:
                if k not in punc and k!=' ' :
                    tmp += k
            str1=tmp.lower()
            tmp = ""
            for k in str2:
                if k not in punc and k!=' ' :
                    tmp += k
            str2=tmp.lower()
            ans.append(str1+" "+str2)
    return ans

#function to generate trigrams
def tri_grams(sentence):
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    l = re.split(r' [\.\?!][\'"\)\]] *', sentence)
  #print(l)
  #return l
    if len(l[len(l)-1])==0:
        l.pop()
  #return l 
    ans=[]
    for i in l:
        kish=list(i.split())
        for j in range(2,len(kish)):
            str1=kish[j-2]
            str2=kish[j-1]
            str3=kish[j]
            tmp = ""
            for k in str1:
                if k not in punc and k!=' ' :
                    tmp += k
            str1=tmp.lower()
            tmp = ""
            for k in str2:
                if k not in punc and k!=' ' :
                    tmp += k
            str2=tmp.lower()
            tmp = ""
            for k in str3:
                if k not in punc and k!=' ' :
                    tmp += k
            str3=tmp.lower()
            ans.append(str1+" "+str2+" "+str3)
    return ans



# The number of top level pages (a top level page is  one that has a link on the home page)
def numberoftoplevelpages(website):
    homepage=findhomepage(website)
    unique_links=getandstorelinks(homepage)
    return(len(unique_links))


#Page titles of all the top level pages 
def topleveltitle(website):
    homepage=findhomepage(website)
    unique_links=getandstorelinks(homepage)
    titles=[]
    for i in unique_links:
        if homepage in i:
            source = urllib.request.urlopen(i).read()
            soup = bs.BeautifulSoup(source,'lxml')
            titles.append(soup.title)
    return titles


#Meta tags from all top level pages
def meta_tags(website):
    homepage=findhomepage(website)
    unique_links=getandstorelinks(homepage)
    metatags=[]
    for i in unique_links:
        if i and homepage in i:
            source = urllib.request.urlopen(i).read()
            soup = bs.BeautifulSoup(source,'lxml')
            metatags.append(soup.find('meta'))
    return metatags

#All internal links (links that point to some page in the same website)
def internal_links(website):
    homepage=findhomepage(website)
    unique_links=getandstorelinks(homepage)
    internallinks=[]
    for i in unique_links:
        if  i and homepage in i:             #checking whether the website name is in the link
            internallinks.append(i)
    return internallinks

#All external links (links that point to pages outside the site). 
def external_links(website):
    homepage=findhomepage(website)
    unique_links=getandstorelinks(homepage)
    externallinks=[]
    for i in unique_links:
        if  i and homepage not  in i:                 #checking whether the website name is not in the link
            externallinks.append(i)
    return externallinks    

#A frequency table of the top 20 key terms (unigrams and bigrams) in the descending order of frequency and write out a CSV file.
def findtop20(ngram):
    d={}
    top20=[]
    for i in ngram:
        if i not in d:
            d[i]=1
        else:
            d[i]+=1
    sorteddict = sorted(d, key=d.get, reverse=True)             #sorting the dictionary based on keys in descending order
    for i in range(20):
        top20.append(sorteddict[i])
    return top20

#Find the size of the web page
def findsizeofwebpage(website):
    r = urlopen(website)
    return len(r.read())

print('\033[1m' +"Enter the website URL")                          #'\033[1m' is used to print the text bold in python
website_name=input()
homepage_name=findhomepage(website_name)

linksinhomepage=[]
linksinhomepage=getandstorelinks(website_name)

textineachpage=getwebpageandstoretext(website_name)

clear_sentence=removepunctuationsandstopwords(website_name)
unigrams=uni_grams(clear_sentence)
bigrams=bi_grams(clear_sentence)
trigrams=tri_grams(clear_sentence)

number_of_top_level_pages=numberoftoplevelpages(website_name)

titles_list=topleveltitle(website_name)

meta_tags_list=meta_tags(website_name)

internal_links_list=internal_links(website_name)

external_links_list=external_links(website_name)

top20unigrams=findtop20(unigrams)
top20bigrams=findtop20(bigrams)
top20trigrams=findtop20(trigrams)

sizeofwebpage=findsizeofwebpage(website_name)



Enter the website URL
https://www.exeterpremedia.com/


In [42]:
#printing the output in a csv file

dictionary={"Homepage": homepage_name, 
            "Links in homepage":linksinhomepage,
            "Text in each link" :textineachpage,
            "Unigrams" : unigrams,
            "Bigrams" : bigrams,
            "Trigrams" : trigrams,
            "Number of top level pages":number_of_top_level_pages,
            "Page titles" :titles_list,
            "Meta tags" : meta_tags_list,
            "All internal links" : internal_links_list,
            "All external links" : external_links_list,
           "Size of web page" : sizeofwebpage}

df=pd.DataFrame.from_dict(dictionary, orient='index')

df.to_csv("featuresandoutput.csv")

In [43]:
#printing the output in a text file (output.txt)

print("INPUT SITE",file=open("output.txt", "a"))
print(website_name,file=open("output.txt", "a"))

print("\nHOME PAGE NAME",file=open("output.txt", "a"))
print(homepage_name,file=open("output.txt", "a"))

print("\nLINKS IN HOME PAGE",file=open("output.txt", "a"))
print(*linksinhomepage,sep="\n",file=open("output.txt", "a"))

# 1.The number of top level pages ((a top level page is  one that has a link on the home page)
print("\nNUMBER OF TOP LEVEL PAGES", file=open("output.txt", "a"))
print(number_of_top_level_pages, file=open("output.txt", "a"))

# 2.Page titles of all the top level pages 
print("\nPAGE TITLES OF ALL TOP LEVEL PAGES", file=open("output.txt", "a"))
print(*titles_list,sep="\n", file=open("output.txt", "a"))

# 3.Meta tags from all top level pages
print("\nMETA TAGS OF TOP LEVEL PAGES", file=open("output.txt", "a"))
print(*meta_tags_list,sep="\n", file=open("output.txt", "a"))

# 4. All internal links (links that point to some page in the same website) 
print("\nALL INTERNAL LINKS", file=open("output.txt", "a"))
print(*internal_links_list,sep="\n", file=open("output.txt", "a"))

# 5. All external links (links that point to pages outside the site).
print("\nALL EXTERNAL LINKS", file=open("output.txt", "a"))
print(*external_links_list,sep="\n", file=open("output.txt", "a"))


# 6. A frequency table of the top 20 key terms (unigrams and bigrams) in the descending order of frequency and write out a CSV file.
print("\nTOP 20 UNIGRAMS", file=open("output.txt", "a"))
print(*top20unigrams,sep="\n", file=open("output.txt", "a"))

print("\nTOP 20 BIGRAMS", file=open("output.txt", "a"))
print(*top20bigrams,sep="\n", file=open("output.txt", "a"))

print("\nTOP 20 TRIGRAMS", file=open("output.txt", "a"))
print(*top20trigrams,sep="\n", file=open("output.txt", "a"))

#frequency table in a csv file
top20dict={"TOP 20 UNIGRAMS" : top20unigrams, "TOP 20 BIGRAMS" : top20bigrams , "TOP 20 TRIGRAMS" : top20trigrams}
df1=pd.DataFrame(top20dict) 
df1.to_csv("top20ngrams.csv")

# Display the size of the webpage
print("\nSIZE OF WEB PAGE", file=open("output.txt", "a"))
print(sizeofwebpage, file=open("output.txt", "a"))


In [44]:
#printing the ouput in jupyter notebook


print('\033[1m' +"INPUT SITE")
print(website_name)

print('\033[1m' +"\nHOME PAGE NAME")
print(homepage_name)

print('\033[1m' +"\nLINKS IN HOME PAGE")
print(*linksinhomepage,sep="\n")

print('\033[1m' +"\nTEXT AFTER REMOVING PUNCTUATIONS AND STOP WORDS")
print(clear_sentence)

print('\033[1m' +"\nUNIGRAMS")
print(*unigrams,)

print('\033[1m' +"\nBIGRAMS")
print(*bigrams)

print('\033[1m' +"\nTRIGRAMS")
print(*trigrams)

print('\033[1m' +"\nNUMBER OF TOP LEVEL PAGES")
print(number_of_top_level_pages)

print('\033[1m' +"\nPAGE TITLES OF ALL TOP LEVEL PAGES")
print(*titles_list,sep="\n")

print('\033[1m' +"\nMETA TAGS OF TOP LEVEL PAGES")
print(*meta_tags_list,sep="\n")

print('\033[1m' +"\nALL INTERNAL LINKS")
print(*internal_links_list,sep="\n")

print('\033[1m' +"\nALL EXTERNAL LINKS")
print(*external_links_list,sep="\n")

print('\033[1m' +"\nTOP 20 UNIGRAMS")
print(*top20unigrams,sep="\n")

print('\033[1m' +"\nTOP 20 BIGRAMS")
print(*top20bigrams,sep="\n")

print('\033[1m' +"\nTOP 20 TRIGRAMS")
print(*top20trigrams,sep="\n")

print('\033[1m' +"\nSIZE OF WEB PAGE")
print(sizeofwebpage)

INPUT SITE
https://www.exeterpremedia.com/

HOME PAGE NAME
https://www.exeterpremedia.com

LINKS IN HOME PAGE
https://www.kriyadocs.com/
https://www.exeterpremedia.com/services/
https://www.exeterpremedia.com/services/#Editorial_Services
https://www.exeterpremedia.com/services/#Data_Services
https://www.exeterpremedia.com/services/#Artwork
https://www.exeterpremedia.com/services/#Project_Management
https://www.exeterpremedia.com/blog/
https://www.exeterpremedia.com/about-us/
https://www.exeterpremedia.com/events/
https://www.exeterpremedia.com/news/
https://www.exeterpremedia.com/contact-us/
https://careers.exeterpremedia.com/jobs/Careers
https://www.exeterpremedia.com/
https://www.exeterpremedia.com/wp-content/uploads/2019/06/atma-global-inc.webp
https://www.exeterpremedia.com/wp-content/uploads/2019/06/bmj-group.webp
https://www.exeterpremedia.com/wp-content/uploads/2019/06/bolinda-publishing-llc.webp
https://www.exeterpremedia.com/wp-content/uploads/2019/06/brill.webp
https://www.ex

Exeter Happy Authors Delighted Publishers var ajaxurl https wwwexeterpremediacomwpadminadminajaxphp body html width100 height100 margin0 padding0 pagepreloader top0 left0 zindex999 position fixed height100 width100 textalign center preloaderpreviewarea webkitanimationdelay 2s animationdelay 2s top50 webkittransform translateY 100 mstransform translateY 100 transform translateY 100 margintop10px maxheight calc 50 20px opacity1 width100 textalign center position absolute preloaderlogo maxwidth90 top50 webkittransform translateY 100 mstransform translateY 100 transform translateY 100 margin 10px auto 0 auto maxheight calc 50 20px opacity1 position relative ballpulse div width15px height15px borderradius100 margin2px webkitanimationfillmode animationfillmode display inlineblock webkitanimation ballpulse 75s infinite cubicbezier 2 68 18108 animation ballpulse 75s infinite cubicbezier 2 68 18108 ballpulse div nthchild 1 webkitanimationdelay 36s animationdelay 36s ballpulse div nthchild 2 web

Kriya cloudbased publishing platform endtoend publishing services Exeter windowabb php windowPHP PHPajax https wwwexeterpremediacomwpadminadminajaxphp PHPwp_p_id 14 var mk_header_parallax mk_banner_parallax mk_page_parallax mk_footer_parallax mk_body_parallax var mk_images_dir https wwwexeterpremediacomwpcontentthemesjupiterassetsimages mk_theme_js_path https wwwexeterpremediacomwpcontentthemesjupiterassetsjs mk_theme_dir https wwwexeterpremediacomwpcontentthemesjupiter mk_captcha_placeholder Enter Captcha mk_captcha_invalid_txt Invalid Try mk_captcha_correct_txt Captcha correct mk_responsive_nav_width1140 mk_vertical_header_back Back mk_vertical_header_anim 1 mk_check_rtltrue mk_grid_width1140 mk_ajax_search_option disable mk_preloader_bg_color fff mk_accent_color fee62c mk_go_to_top true mk_smooth_scroll true mk_preloader_bar_color fee62c mk_preloader_logo var mk_header_parallaxfalse mk_banner_parallaxfalse mk_footer_parallaxfalse mk_body_parallaxfalse mk_no_more_posts No More Posts 

 3619 3626 3623 3619 3619 3588 3660 TH80 Nakhon Si Thammarat 3609 3588 3619 3624 3619 3637 3608 3619 3619 3617 3619 3634 3594 TH55 Nan 3609 3656 3634 3609 TH96 Narathiwat 3609 3619 3634 3608 3636 3623 3634 3626 TH39 Nong Bua Lam Phu 3627 3609 3629 3591 3610 3633 3623 3621 3635 3616 3641 TH43 Nong Khai 3627 3609 3629 3591 3588 3634 3618 TH12 Nonthaburi 3609 3609 3607 3610 3640 3619 3637 TH13 Pathum Thani 3611 3607 3640 3617 3608 3634 3609 3637 TH94 Pattani 3611 3633 3605 3605 3634 3609 3637 TH82 Phang Nga 3614 3633 3591 3591 3634 TH93 Phatthalung 3614 3633 3607 3621 3640 3591 TH56 Phayao 3614 3632 3648 3618 3634 TH67 Phetchabun 3648 3614 3594 3619 3610 3641 3619 3603 3660 TH76 Phetchaburi 3648 3614 3594 3619 3610 3640 3619 3637 TH66 Phichit 3614 3636 3592 3636 3605 3619 TH65 Phitsanulok 3614 3636 3625 3603 3640 3650 3621 3585 TH54 Phrae 3649 3614 3619 3656 TH83 Phuket 3616 3641 3648 3585 3655 3605 TH25 Prachin Buri 3611 3619 3634 3592 3637 3609 3610 3640 3619 3637 TH77 Prachuap Khiri Kh

 SV Suceava TR Teleorman TM Timi 537 TL Tulcea VL V acirc lcea VS Vaslui VN Vrancea ES C A Coru ntilde VI Araba Aacute lava AB Albacete A Alicante AL Almer iacute O Asturias AV Aacute vila BA Badajoz PM Baleares B Barcelona BU Burgos CC C aacute ceres CA C aacute diz S Cantabria CS Castell oacute n CE Ceuta CR Ciudad Real CO C oacute rdoba CU Cuenca GI Girona GR Granada GU Guadalajara SS Gipuzkoa H Huelva HU Huesca J Ja eacute n LO La Rioja GC Las Palmas LE Le oacute n L Lleida LU Lugo M Madrid MA M aacute laga ML Melilla MU Murcia NA Navarra OR Ourense P Palencia PO Pontevedra SA Salamanca TF Santa Cruz de Tenerife SG Segovia SE Sevilla SO Soria T Tarragona TE Teruel TO Toledo V Valencia VA Valladolid BI Bizkaia ZA Zamora Z Zaragoza CA AB Alberta BC British Columbia MB Manitoba NB New Brunswick NL Newfoundland Labrador NT Northwest Territories NS Nova Scotia NU Nunavut ON Ontario PE Prince Edward Island QC Quebec SK Saskatchewan YT Yukon Territory NZ NL Northland AK Auckland WA Waikat

...

 yalova tr78 karab tr78 karab 252 karab 252 k 252 k tr79 k tr79 kilis tr79 kilis tr80 kilis tr80 osmaniye tr80 osmaniye tr81 osmaniye tr81 d tr81 d 252 d 252 zce 252 zce hk zce hk hong hk hong kong hong kong hong kong hong kong hong kong island kong island kowloon island kowloon kowloon kowloon kowloon new kowloon new territories new territories new territories new territories new territories var territories var ajaxurlhttps var ajaxurlhttps wwwexeterpremediacomwpadminadminajaxphp ajaxurlhttps wwwexeterpremediacomwpadminadminajaxphp var wwwexeterpremediacomwpadminadminajaxphp var msgrabbingcurosrhttps var msgrabbingcurosrhttps wwwexeterpremediacomwpcontentpluginsmastersliderpublicassetscsscommongrabbingcur msgrabbingcurosrhttps wwwexeterpremediacomwpcontentpluginsmastersliderpublicassetscsscommongrabbingcur msgrabcurosrhttps wwwexeterpremediacomwpcontentpluginsmastersliderpublicassetscsscommongrabbingcur msgrabcurosrhttps wwwexeterpremediacomwpcontentpluginsmastersliderpublicassetscssc

solid rgba 000 rgba 000 06 000 06 border1px 06 border1px solid border1px solid fff solid fff important fff important padding16px important padding16px important padding16px important boxshadow1px important boxshadow1px 1px boxshadow1px 1px 8px 1px 8px 1px 8px 1px adadad 1px adadad borderradius4px adadad borderradius4px mktweetlist borderradius4px mktweetlist li mktweetlist li mksvgicon li mksvgicon position mksvgicon position absolute position absolute height16px absolute height16px top height16px top 13px top 13px left 13px left 7px left 7px mkblognewspaperitem 7px mkblognewspaperitem blogitemholder mkblognewspaperitem blogitemholder margin0 blogitemholder margin0 8px margin0 8px 23px 8px 23px border1px 23px border1px solid border1px solid fff solid fff backgroundcolor fff backgroundcolor fff backgroundcolor fff position fff position relative position relative boxshadow0 relative boxshadow0 2px boxshadow0 2px 1px 2px 1px 0 1px 0 rgba 0 rgba 000 rgba 000 05 000 05 boxshadow1px 05 boxsh

 0a 7d 0acustom 7d 0acustom 20tpbullet 0acustom 20tpbullet 20 20tpbullet 20 7b 20 7b 0a 7b 0a 09width 0a 09width 3a12px 09width 3a12px 3b 3a12px 3b 0a 3b 0a 09height 0a 09height 3a12px 09height 3a12px 3b 3a12px 3b 0a 3b 0a 09position 0a 09position 3aabsolute 09position 3aabsolute 3b 3aabsolute 3b 0a 3b 0a 09background 0a 09background 3a 09background 3a 23aaa 3a 23aaa 3b 23aaa 3b 0a 3b 0a 20 0a 20 20 20 20 20 20 20 20background 20 20background 3argba 20background 3argba 28125 3argba 28125 2c125 28125 2c125 2c125 2c125 2c125 2c05 2c125 2c05 29 2c05 29 3b 29 3b 0a 3b 0a 09cursor 0a 09cursor 3a 09cursor 3a 20pointer 3a 20pointer 3b 20pointer 3b 0a 3b 0a 09boxsizing 0a 09boxsizing 3acontentbox 09boxsizing 3acontentbox 3b 3acontentbox 3b 0a 3b 0a 7d 0a 7d 0acustom 7d 0acustom 20tpbullet 0acustom 20tpbullet 3ahover 20tpbullet 3ahover 2c 3ahover 2c 0acustom 2c 0acustom 20tpbulletselected 0acustom 20tpbulletselected 20 20tpbulletselected 20 7b 20 7b 0a 7b 0a 09background 0a 09background 3argb 0

 errormessage span stylefontsize16px span stylefontsize16px color stylefontsize16px color bc0c06 color bc0c06 errormessage bc0c06 errormessage span errormessage span jquery span jquery sliderid jquery sliderid show sliderid show html show html errormessage html errormessage cdata errormessage cdata var cdata var wpcf7 var wpcf7 apisettings wpcf7 apisettings root apisettings root https root https wwwexeterpremediacomwpjsoncontactform7v1 https wwwexeterpremediacomwpjsoncontactform7v1 namespace wwwexeterpremediacomwpjsoncontactform7v1 namespace contactform7v1 namespace contactform7v1 cdata contactform7v1 cdata var cdata var leadinwordpress var leadinwordpress userrole leadinwordpress userrole visitor userrole visitor pagetype visitor pagetype home pagetype home leadinpluginversion home leadinpluginversion 7281 leadinpluginversion 7281 cdata 7281 cdata var cdata var wpcf7redirectforms var wpcf7redirectforms 2194 wpcf7redirectforms 2194 pageid 2194 pageid 0 pageid 0 externalurl 0 externalur

https://www.exeterpremedia.com/2018/02/
https://www.exeterpremedia.com
https://www.exeterpremedia.com/exeter-privacy-policy/
https://www.exeterpremedia.com/terms-and-conditions/

ALL EXTERNAL LINKS
https://www.kriyadocs.com/
https://careers.exeterpremedia.com/jobs/Careers
http://twitter.com//statuses/
https://twitter.com/Exeter_Premedia
https://www.linkedin.com/in/parivalavan-manoharan/
https://www.linkedin.com/in/jaishankarn/
https://www.linkedin.com/in/augustine-k/
https://www.linkedin.com/feed/hashtag/?keywords=%23ExeterPremedia
https://www.linkedin.com/feed/hashtag/?keywords=%23Exeter
https://www.linkedin.com/feed/hashtag/?keywords=%23Kriya
https://www.linkedin.com/feed/hashtag/?keywords=%23HappyAuthors
https://www.linkedin.com/feed/hashtag/?keywords=%23SVCE
https://www.linkedin.com/feed/hashtag/?keywords=%23YetAnotherHackathon
https://www.linkedin.com/feed/hashtag/?keywords=%23YAH
https://www.linkedin.com/feed/hashtag/?keywords=%23Acmsvce
https://www.linkedin.com/feed/hashtag/?key